# Numerical differentiation of noisy data: Ground truth results

This notebook builds on `1_basic_tutorial.ipynb` by calling optimization routines to find optimal parameters for each method.

Some notes on notation and syntax:
  * dt: (float) time step (note: we concentrated on differentiating timeseries data, however these functions work for a 1-dimensional spatial derivative as well)
  * x: (np.array with length N) the time series you want to differentiate
  * x_hat: (np.array like x) the smoothed estimate of x
  * x_truth: (np.array like x) the true value of x (which is known when the data is simulated, and used for plotting purposes only)
  * dxdt_hat: (np.array like x) the estimate of the derivative of x
  * dxdt_truth: (np.array like x) the true value of dxdt (which is known when the data is simulated, and used for plotting purposes only)

In [ ]:
import os
import sys
import time
import numpy as np

# local import
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pynumdiff

simulate = pynumdiff.utils.simulate
evaluate = pynumdiff.utils.evaluate

%matplotlib inline

# 0. Simulate some data

pynumdiff comes with 6 different toy problems (choose the one that most resembles your data to see how the methods compare):
* lorenz_x: (nonlinear) x component of a lorenz attractor
* sine: (linear) sum of two sines
* pop_dyn: (nonlinear) bounded exponential growth
* triangle: (nonlinear) sharp-edged triangle wave with increasing frequency
* pi_control: (linear / nonlinear) linear proportional integral controller with nonlinear control inputs

In [ ]:
# simulation parameters

# noise is generated using np.random, e.g. np.random.normal, np.random.uniform, np.random.poisson
# noise_type and noise_parameters should be compatible with np.random functions 
noise_type = 'normal'
noise_parameters = [0, 0.01]

# time step and time series length
dt = 0.1 # sampling time step
simdt = 0.01 # simulation timestep
timeseries_length =  50 # sec
problem = 'pi_control'

In [ ]:
# simulate data

x, x_truth, dxdt_truth, extras = r = pynumdiff.utils.simulate.__dict__[problem](timeseries_length, 
                                                                                noise_parameters=noise_parameters, 
                                                                                dt=dt, 
                                                                                simdt=0.01)

In [ ]:
# For our toy problems we can optimize the parameters in one of two options:
# 1. Use the known dxdt to find the best parameters 
# 2. Actual dxdt is unknown

# In the real world, option (1) is not possible. 

# This notebook allows you to run the optimization with either option:
# cheat=True selects option (1)
# cheat=False selects option (2)

cheat = True

if cheat:
    dxdt_truth_vals = dxdt_truth
else:
    dxdt_truth_vals = None

The optimization approach described here solves a loss function that balances the faithfulness and smoothness of the derivative estimate, and relies on a single hyperparameter, $\gamma$, or `tvgamma` in the code. See the paper for more detail, but a brief overview is given here.

The loss function to find the optimal parameters is as follows,

$$L = \mbox{RMSE} \bigg( \mbox{trapz}(\mathbf{\hat{\dot{x}}}(\Phi)) + \mu, \mathbf{y} \bigg) + \gamma \bigg({TV}\big(\mathbf{\hat{\dot{x}}}(\Phi)\big)\bigg),$$
where $\mathbf{y}$ are the noisy measurements, \mathbf{\hat{\dot{x}}} is the estimate of the derivative, $\mbox{trapz}(\cdot)$ is the discrete-time trapezoidal numerical integral, $\mu$ resolves the unknown integration constant, \\(\gamma\\) is a hyper-parameter, and $TV$ is the total variation,

$$TV(\mathbf{\hat{\dot{x}}}) = \frac{1}{m}\left\lVert\mathbf{\hat{\dot{x}}}_{0:m-1}-\mathbf{\hat{\dot{x}}}_{1:m}\right\rVert_{1}.$$

To use the code to  solve the loss function and find your optimal parameters is quite simple.

First, find tvgamma using the following heuristic: 

In [ ]:
cutoff_frequency = 0.1 # high frequency of signal in the data
log_gamma = -1.6*np.log(cutoff_frequency) -0.71*np.log(dt) - 5.1
tvgamma = np.exp(log_gamma)      

Then, use the following syntax, as detailed in the notebook.

    params, val = pynumdiff.optimize.sub_module.method(x, dt, params=None, 
                                                   tvgamma=tvgamma, # hyperparameter
                                                   dxdt_truth=None, # no ground truth data
                                                   options={})
    print('Optimal parameters: ', params)
    x_hat, dxdt_hat = pynumdiff.sub_module.method(x, dt, params, options={'smooth': True})

In [ ]:
if cheat: # so the code doesn't break
    tvgamma = 0 # since we are optimizing on the actual values, no need to regularize
    
print('tvgamma = ', tvgamma)

## 1. Finite Difference

### 1.1 Finite Difference: First Order

In [ ]:
x_hat, dxdt_hat = pynumdiff.finite_difference.first_order(x, dt)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 1.2 Finite Difference: Second Order

In [ ]:
x_hat, dxdt_hat = pynumdiff.finite_difference.first_order(x, dt)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 1.3 Finite Difference: Iterated First Order

In [ ]:
params, val = pynumdiff.optimize.finite_difference.first_order(x, dt, params=None, 
                                                               options={'iterate': True},
                                                               tvgamma=tvgamma,
                                                               dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.finite_difference.first_order(x, dt, params, options={'iterate': True})
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

## 2. Smooth Finite Difference

### 2.1 Smooth Finite Difference: Median smoothing

In [ ]:
params, val = pynumdiff.optimize.smooth_finite_difference.mediandiff(x, dt, params=None, 
                                                                     options={'iterate': True},
                                                                     tvgamma=tvgamma,
                                                                     dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.smooth_finite_difference.mediandiff(x, dt, params, options={'iterate': True})
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 2.2 Smooth Finite Difference: Mean smoothing

In [ ]:
params, val = pynumdiff.optimize.smooth_finite_difference.meandiff(x, dt, params=None, 
                                                                   options={'iterate': True},
                                                                   tvgamma=tvgamma,
                                                                     dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.smooth_finite_difference.meandiff(x, dt, params, options={'iterate': True})
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 2.3 Smooth Finite Difference: Gaussian smoothing

In [ ]:
params, val = pynumdiff.optimize.smooth_finite_difference.gaussiandiff(x, dt, params=None, 
                                                                       options={'iterate': True},
                                                                       tvgamma=tvgamma,
                                                                     dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.smooth_finite_difference.gaussiandiff(x, dt, params, options={'iterate': True})
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 2.4 Smooth Finite Difference: Friedrichs smoothing

In [ ]:
params, val = pynumdiff.optimize.smooth_finite_difference.friedrichsdiff(x, dt, params=None, 
                                                                         options={'iterate': True},
                                                                         tvgamma=tvgamma,
                                                                     dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.smooth_finite_difference.friedrichsdiff(x, dt, params, options={'iterate': True})
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 2.5 Smooth Finite Difference: Butterworth smoothing

In [ ]:
params, val = pynumdiff.optimize.smooth_finite_difference.butterdiff(x, dt, params=None, 
                                                                     options={'iterate': True},
                                                                     tvgamma=tvgamma,
                                                                     dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.smooth_finite_difference.butterdiff(x, dt, params, options={'iterate': True})
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 2.6 Smooth Finite Difference: Spline smoothing

In [ ]:
params, val = pynumdiff.optimize.smooth_finite_difference.splinediff(x, dt, params=None, 
                                                                     options={'iterate': True},
                                                                     tvgamma=tvgamma,
                                                                     dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.smooth_finite_difference.splinediff(x, dt, params, options={'iterate': True})
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

## 3. Total Variation Regularization

### 3.1 Iterative Total Variation Regularization (regularized velocity)

In [ ]:
params, val = pynumdiff.optimize.total_variation_regularization.iterative_velocity(x, dt, params=None, 
                                                                                   tvgamma=tvgamma,
                                                                                   dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.total_variation_regularization.iterative_velocity(x, dt, params)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 3.2 Convex Total Variation Regularization: regularize velocity

In [ ]:
params, val = pynumdiff.optimize.total_variation_regularization.velocity(x, dt, params=None, 
                                                                         tvgamma=tvgamma,
                                                                         dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.total_variation_regularization.velocity(x, dt, params)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 3.3 Convex Total Variation Regularization: regularize acceleration

In [ ]:
params, val = pynumdiff.optimize.total_variation_regularization.acceleration(x, dt, params=None, 
                                                                             tvgamma=tvgamma,
                                                                             dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.total_variation_regularization.acceleration(x, dt, params)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 3.4 Convex Total Variation Regularization: regularize jerk

In [ ]:
params, val = pynumdiff.optimize.total_variation_regularization.jerk(x, dt, params=None, 
                                                                     tvgamma=tvgamma,
                                                                    dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.total_variation_regularization.jerk(x, dt, params)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 3.5 Convex Total Variation Regularization: regularize acceleration with gaussian smoothing

In [ ]:
params, val = pynumdiff.optimize.total_variation_regularization.smooth_acceleration(x, dt, params=None, 
                                                                             tvgamma=tvgamma,
                                                                             dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.total_variation_regularization.smooth_acceleration(x, dt, params)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

## 4. Linear Models

### 4.1 Linear Models: Spectral derivative

In [ ]:
params, val = pynumdiff.optimize.linear_model.spectraldiff(x, dt, params=None, 
                                                           tvgamma=tvgamma,
                                                                     dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.linear_model.spectraldiff(x, dt, params)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 4.2 Linear Models: Sliding polynomial fit

In [ ]:
params, val = pynumdiff.optimize.linear_model.polydiff(x, dt, params=None, 
                                                       tvgamma=tvgamma,
                                                                     dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.linear_model.polydiff(x, dt, params)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 4.3 Linear Models: Savitzky-Golay filter

In [ ]:
params, val = pynumdiff.optimize.linear_model.savgoldiff(x, dt, params=None, 
                                                         tvgamma=tvgamma,
                                                         dxdt_truth=dxdt_truth_vals,
                                                         options={'smooth': True})
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.linear_model.savgoldiff(x, dt, params, options={'smooth': True})
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 4.4 Linear Models: Sliding chebychev polynomial fit

In [ ]:
params, val = pynumdiff.optimize.linear_model.chebydiff(x, dt, params=None, 
                                                        tvgamma=tvgamma,
                                                                     dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.linear_model.chebydiff(x, dt, params)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 4.5 Linear Models: Sliding dynamic mode decomposition fit

In [ ]:
if 0: # this one is too slow!
    params, val = pynumdiff.optimize.linear_model.dmddiff(x, dt, params=None, 
                                                           tvgamma=tvgamma,
                                                          dxdt_truth=dxdt_truth_vals)
    print('Optimal parameters: ', params)
    x_hat, dxdt_hat = pynumdiff.linear_model.dmddiff(x, dt, params)
    evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 4.6 Linear Models: Sliding linear time invariant system fit

In [ ]:
if 0: # this one is too slow!
    params, val = pynumdiff.optimize.linear_model.lineardiff(x, dt, params=None, 
                                                             tvgamma=tvgamma,
                                                                         dxdt_truth=dxdt_truth_vals)
    print('Optimal parameters: ', params)
    x_hat, dxdt_hat = pynumdiff.linear_model.lineardiff(x, dt, params)
    evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

## Kalman smoothing

### 5.1 Kalman smoothing: constant velocity (forward-backward)

In [ ]:
params, val = pynumdiff.optimize.kalman_smooth.constant_velocity(x, dt, params=None, 
                                                                 tvgamma=tvgamma,
                                                                     dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.kalman_smooth.constant_velocity(x, dt, params)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 5.2 Kalman smoothing: constant acceleration (forward-backward)

In [ ]:
params, val = pynumdiff.optimize.kalman_smooth.constant_acceleration(x, dt, params=None, 
                                                                 tvgamma=tvgamma,
                                                                    dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.kalman_smooth.constant_acceleration(x, dt, params)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)

### 5.3 Kalman smoothing: constant jerk (forward-backward)

In [ ]:
params, val = pynumdiff.optimize.kalman_smooth.constant_jerk(x, dt, params=None, 
                                                                 tvgamma=tvgamma,
                                                                     dxdt_truth=dxdt_truth_vals)
print('Optimal parameters: ', params)
x_hat, dxdt_hat = pynumdiff.kalman_smooth.constant_jerk(x, dt, params)
evaluate.plot(x, dt, x_hat, dxdt_hat, x_truth, dxdt_truth)